## pg_restore cipsrv_support

This notebook will download and load (or reload) the cipsrv_support schema into a cipsrv database. 

In [ ]:
import os,sys,psycopg2,argparse;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
from contextlib import closing;
parser = argparse.ArgumentParser();
parser.add_argument("--support_dumpfile" ,required=False);
parser.add_argument("--use_existing"     ,required=False,default=False,action=argparse.BooleanOptionalAction);
parser.add_argument("--override_username",required=False);
args, _ = parser.parse_known_args();

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

s3bk = os.environ['S3_BUCKET'];
# override: s3bk = 'https://dmap-data-commons-ow.s3.amazonaws.com/data/cipsrv/';
lddk = os.path.join(os.path.expanduser('~'),'loading_dock');
dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pswd = os.environ['POSTGRESQL_CIP_PASS'];
thds = 2;

component_name = 'cipsrv_support';
component_type = 'dataset';
if args.support_dumpfile is None:
   src = 'cipsrv_support_1.dmp';
else:
   src = args.support_dumpfile;

print("source: " + s3bk + src);

Download the dump file from S3

In [ ]:
if args.use_existing:
    print("using existing " + src);

else:
    z = common.download(
        s3  = s3bk
       ,src = src
       ,ld  = lddk
       ,trg = src
    );

    if not os.path.exists(os.path.join(lddk,src)):
        raise Exception('failed to download dmp file');

    print("successfully downloaded " + src);
    

Ping the database to make sure connectivity is up

In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
with closing(conn.cursor()) as cursor:
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha_3338 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha_5070 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha_26904 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha_32161 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha_32655 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_aiannha_32702 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters_3338 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters_5070 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters_26904 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters_32161 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters_32655 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tiger_fedstatewaters_32702 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.tribal_crosswalk CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.wbd_hu12_f3   CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.wbd_hu12_np21 CASCADE");
    cursor.execute("DROP TABLE IF EXISTS cipsrv_support.wbd_hu12_nphr CASCADE");
    conn.commit();

print("preexisting data purged.");

Execute pg_restore command line tool to load the download into the database

In [ ]:
z = common.pg_restore(
     host
    ,port
    ,dbse
    ,user
    ,pswd
    ,os.path.join(lddk,src)
    ,thds
);

print("pg_restore complete.");

Inspect the results to verify no serious errors occurred duing import

In [ ]:
if z[0] == b'':
    a = common.upsert_registry(conn,component_name,component_type,src,args.override_username);
else:
    print(str(z));
    sys.exit(-10);
